## Reading Data

In [ ]:
# Library Imports.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Allows plots to appear directly in the notebook.
%matplotlib inline

from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.utils import resample
from sklearn.model_selection import KFold

import sqlalchemy as sqla
import pymysql
from sqlalchemy import create_engine

import csv
import datetime as dt

In [ ]:
URI='database-comp30830.c2kwpm1jk01q.us-east-1.rds.amazonaws.com'
PORT='3306'
DB='comp30830_db'
PASSWORD='Simple12'
USER='admin'

In [ ]:
engine = create_engine("mysql+mysqldb://{}:{}@{}:{}/{}".format(USER, PASSWORD,
                                                              URI, PORT, DB), echo=True)

In [ ]:
bikes=pd.read_sql_table('live_bike_data', engine)  

In [ ]:
# Make a new dataframe of this table
bikes.to_csv('allBikes.csv', index=False)

In [ ]:
# Read csv file into a dataframe.
bikes = pd.read_csv('allBikes.csv')

In [ ]:
weather=pd.read_sql_table('live_weather_data', engine)  

In [ ]:
# Make a new dataframe of this table
weather.to_csv('allWeather.csv', index=False)

In [ ]:
# Read csv file into a dataframe.
weather = pd.read_csv('allWeather.csv')

In [ ]:
bikes['datetime'] = pd.to_datetime(bikes['date'] + ' ' + bikes['time'])
weather['datetime'] = pd.to_datetime(weather['date'] + ' ' + weather['time'])

In [ ]:
bikes = bikes.sort_values(by='datetime')
weather = weather.sort_values(by='datetime')

## This can be used as a checkpoint, start from here if you want to run again without having to call from the database.

In [ ]:
full_df = pd.merge_asof(bikes, weather, left_on="datetime", right_on="datetime",direction="nearest")

In [ ]:
full_df.head(3)

In [ ]:
# constrict the dataframe to only those times in which the service is available to users.
full_df = full_df.drop(full_df[(full_df.datetime.dt.hour > 0) & (full_df.datetime.dt.hour < 5)].index)

In [ ]:
## Create four flags each representing the stage of the day.
morning_start = pd.to_datetime("05:00:00").time()
morning_end = pd.to_datetime("12:00:00").time()
afternoon_start = pd.to_datetime("12:01:00").time()
afternoon_end = pd.to_datetime("16:59:00").time()
evening_start = pd.to_datetime("17:00:00").time()
evening_end = pd.to_datetime("20:00:00").time()
night_start = pd.to_datetime("20:01:00").time()
night_end = pd.to_datetime("23:59:59").time()

In [ ]:
full_df['morning'] = np.where((full_df['datetime'].dt.time > morning_start)
                         & (full_df['datetime'].dt.time < morning_end),
                         1, 0)

full_df['afternoon'] = np.where((full_df['datetime'].dt.time > afternoon_start)
                         & (full_df['datetime'].dt.time < afternoon_end),
                         1, 0)

full_df['evening'] = np.where((full_df['datetime'].dt.time > evening_start)
                         & (full_df['datetime'].dt.time < evening_end),
                         1, 0)

full_df['night'] = np.where((full_df['datetime'].dt.time > night_start)
                         & (full_df['datetime'].dt.time < night_end),
                         1, 0)

In [ ]:
#replace days with numbers
full_df["day_x"].replace(['Mon','Tue','Wed','Thu','Fri','Sat','Sun'], [0,1,2,3,4,5,6], inplace=True)

In [ ]:
# create a time of day column, based on the hours of the day.
full_df['tod'] = full_df.datetime.dt.hour

- Observe the usage on a given weekday at a particular station:

In [ ]:
weekday_test_df = full_df.loc[(full_df['ID'] == 84) & (full_df['day_x'] == 2)]

In [ ]:
# First, plot the observed data
weekday_test_df.plot(kind='scatter', x='tod', y='availableBikes')

In [ ]:
weekday_test_df = full_df.loc[(full_df['ID'] == 84) & (full_df['day_x'] == 4)]

In [ ]:
weekday_test_df.plot(kind='scatter', x='tod', y='availableBikes')

- Observe the usage on a given weekend day at the same station:

In [ ]:
weekend_test_df = full_df.loc[(full_df['ID'] == 84) & (full_df['day_x'] == 5)]

In [ ]:
# First, plot the observed data
weekend_test_df.plot(kind='scatter', x='tod', y='availableBikes')

### From each of these plots, we can observe a variance between usage during the week and during the weekend. Weekdays generally see heavier usage during the morning time, while weekends see more usage during the afternoon/evening time. Therefore, two spearate models will be developed- one for Monday to Friday and another for Saturdays and Sundays.

In [ ]:
#clouds
full_df["number"].replace([801,802,803,804], 'clouds', inplace=True)

#clear
full_df["number"].replace([800], 'clear', inplace=True)

#Atmosphere
full_df["number"].replace([701,711,721,731,741,751,761,762,771,781], 'Atmosphere', inplace=True)

#snow
full_df["number"].replace([600,601,602,611,612,613,615,616,620,621,622], 'snow', inplace=True)

#rain
full_df["number"].replace([500,501,502,503,504,511,520,521,522,531], 'rainfall', inplace=True)

#drizzle
full_df["number"].replace([300,301,302,310,311,312,313,314,321], 'drizzle', inplace=True)

#thunderstorm
full_df["number"].replace([200,201,202,210,211,212,221,230,231,232], 'thunderstorm', inplace=True)

In [ ]:
full_df.head()

In [ ]:
full_df.drop(["date_x","time_x","status", "epoch", "main",
         "description","icon", "tempMin", "tempMax", "tempFeels", "humidity",
         "pressure", "windSpeed","windDeg","sunrise", "sunset",
             "date_y","time_y", "day_y"],axis=1,inplace=True)

In [ ]:
# add a flag that indicates whether a day is dry (has zero rain)
full_df['dry_day'] = (full_df['rain'] == 0).astype(int)

In [ ]:
choice = int(input("Please enter a number to predict for either availableBikes - (0) or availableBikeStands - (1): "))
if choice == 0:
    full_df = full_df.rename(columns={"availableBikes": "target"})
    full_df.drop(["availableBikeStands"], axis=1,inplace=True)    
else:
    full_df = full_df.rename(columns={"availableBikeStands": "target"})
    full_df.drop(["availableBikes"], axis=1,inplace=True)

In [ ]:
week_df = full_df.loc[(full_df['day_x'] >= 0) & (full_df['day_x'] <= 4)]

In [ ]:
weekend_df = full_df.loc[(full_df['day_x'] >= 5) & (full_df['day_x'] <= 6)]

In [ ]:
station = int(input("Please enter station ID: "))
week_or_weekend = int(input("Please choose to predict for week - (0) or weekend - (1): "))
# bikes_or_stands = int(input("Please choose to predict either availableBikeStands(0) or availableBikes(1): "))
# Constrain df to a single station on a single day
if week_or_weekend == 0:
    new_df = week_df.loc[(week_df.ID == station)]
else:
    new_df = weekend_df.loc[(weekend_df.ID == station)]

# Constrain df to a single station on a single day
# new_df = full_df.loc[(full_df['ID'] == station) & (full_df['day_x'] == day)]

In [ ]:
## Dropping all columns not necessary for predictive model.
new_df.drop(["ID", "datetime"], axis=1,inplace=True)

In [ ]:
# Make a new dataframe of this station
new_df.to_csv('comp303830_model_multipleLinearRegression.csv', index=False)

In [ ]:
# Read csv file into a dataframe.
df = pd.read_csv('comp303830_model_multipleLinearRegression.csv')

In [ ]:
df.head(3)

In [ ]:
# Print the average target(availableBikes/availableBikeStands) in our dataset.
# We could use this as a very simple baseline prediction model.
# A better prediction model should at least improve on this baseline model.
round(df.target.mean())

### Observing the data:
- Trying to find correlations between continuous data and the target feature:

In [ ]:
# First, plot the observed data
df.plot(kind='scatter', x='rain', y='target')

In [ ]:
# First, plot the observed data
df.plot(kind='scatter', x='temp', y='target')

- There does not appear to be a clear correlation between the target feature and the continuous data

In [ ]:
## Keep this pandas series for later.
## Will be used below.
tod_placeholder = df[['tod']]

# Training with continuous and categorical features

In [ ]:
#replace days with numbers
df["day_x"].replace([0,1,2,3,4,5,6], ['Mon','Tue','Wed','Thu','Fri','Sat','Sun'], inplace=True)

In [ ]:
#We can also do this directly for all categorical features
df = pd.get_dummies(df, drop_first=True)

In [ ]:
# Input features must exclude the target feature
column_names = list(df.columns)[1:]

In [ ]:
X = df[column_names]
y = df.target

In [ ]:
# drop_first = True removes multi-collinearity
add_var = pd.get_dummies(X['tod'], prefix='tod', drop_first=True)
# Add all the columns to the model data
X = X.join(add_var)
# Drop the original column that was expanded
X.drop(columns=['tod'], inplace=True)

In [ ]:
X.head()

## Train a model based upon multiple linear regression

In [ ]:
# Drop any rows with null values
df.dropna(axis=0, how='any', inplace=True)

model = LinearRegression(fit_intercept=False)
model.fit(X, y)
df['deg_1_pred'] = model.predict(X)

In [ ]:
#This function is used repeatedly to compute all metrics
def printMetrics(testActualVal, predictions):
    #classification evaluation measures
    print('\n==============================================================================')
    print("MAE: ", metrics.mean_absolute_error(testActualVal, predictions))
    #print("MSE: ", metrics.mean_squared_error(testActualVal, predictions))
    print("RMSE: ", metrics.mean_squared_error(testActualVal, predictions)**0.5)
    print("R2: ", metrics.r2_score(testActualVal, predictions))

In [ ]:
printMetrics(y, model.predict(X))

In [ ]:
df[['target', 'deg_1_pred']].plot(alpha=0.5, figsize=(20, 5))

<ref: https://jakevdp.github.io/PythonDataScienceHandbook/05.06-linear-regression.html >

## Use k-folds cross-validation (k=3) to assess the performance of our model:

In [ ]:
X_column_name = list(X.columns)
X_one_deg = pd.DataFrame(X[X_column_name])
y = pd.DataFrame(df.target)
model = LinearRegression()
scores = []
print("Coefficient of Determination (R2):")
kfold = KFold(n_splits=3, shuffle=True, random_state=42)
for i, (train, test) in enumerate(kfold.split(X_one_deg, y)):
    model.fit(X_one_deg.iloc[train,:], y.iloc[train,:])
    score = model.score(X_one_deg.iloc[test,:], y.iloc[test,:])
    scores.append(score)
    print(scores)

<ref: https://becominghuman.ai/linear-regression-in-python-with-pandas-scikit-learn-72574a2ec1a5 >

## Having already observed a nonlinear relationship between bike availability and the time of the day, we will now investigate whether our linear model would more effectively predict our target value if implemented polynomially.

In [ ]:
## Using scikit's built-in Polynomial Features
polynomial_features= PolynomialFeatures(degree=2)
X_poly = polynomial_features.fit_transform(X)

In [ ]:
model = LinearRegression()
model.fit(X_poly, y)
y_poly_pred = model.predict(X_poly)
df['deg_2_pred'] = y_poly_pred

rmse = np.sqrt(mean_squared_error(y,y_poly_pred))
r2 = r2_score(y,y_poly_pred)
print("Root Mean Square Error: ", rmse)
print("Coefficient of Determination (R2): ", r2)

In [ ]:
df[['target', 'deg_2_pred']].plot(alpha=0.5, figsize=(20, 5))

## The model trained on polynomial data appears to offer a marginally improved r^2 value in comparison to the single dimensional based model.

# Train/Testing evaluation will be carried out on each model.

## Evaluation with train/test split - single dimensional & polynomial based models.

In [ ]:
# Split the data into train and test sets
# Take a third (random) data samples as test data, rest as training data
X_train_one_d, X_test_one_d, y_train_one_d, y_test_one_d = train_test_split(X, y, test_size=0.3)
X_train_polynomial, X_test_polynomial, y_train_polynomial, y_test_polynomial = train_test_split(X_poly, y, test_size=0.3)

In [ ]:
# Train on the training sample and test on the test sample.
one_d_multi_linreg = LinearRegression().fit(X_train_one_d, y_train_one_d)
polynomial_multi_linreg = LinearRegression().fit(X_train_polynomial, y_train_polynomial)

In [ ]:
# Predicted avaialbleBikes/availableBikeStands on training set
one_d_train_predictions = one_d_multi_linreg.predict(X_train_one_d)
polynomial_train_predictions = polynomial_multi_linreg.predict(X_train_polynomial)
print("Training metrics for single dimensional model:")
printMetrics(y_train_one_d, one_d_train_predictions)
print(" ")
print("Training metrics for polynomial model:")
printMetrics(y_train_polynomial, polynomial_train_predictions)

### The training metrics for the polynomial model are marinally better than the single-dimensional model.

In [ ]:
# Predicted avaialbleBikes/availableBikeStands on test set
one_d_test_predictions = one_d_multi_linreg.predict(X_test_one_d)
polynomial_test_predictions = polynomial_multi_linreg.predict(X_test_polynomial)

# print("Actual values of one dimensionaltest:\n", y_test_one_d)
# print("Predictions on single dimensional test:", one_d_test_predictions)
print("Testing metrics for single dimensional model:")
printMetrics(y_test_one_d, one_d_test_predictions)
print(" ")
# print("Actual values of polynomial test:\n", y_test_polynomial)
# print("Predictions on polynomial test:", polynomial_test_predictions)
print("Testing metrics for polynomial model:")
printMetrics(y_test_polynomial, polynomial_test_predictions)

### Here we can observe differences in the results produced by each model when trained on data from the days of the the week as opposed to the days of the weekend:
#### The polynomial model, when introduced to unseen data, performs extremely poorly on the days of the week based data. This is likely due to overfitting. For this reason we will discard the polynomial model for our data based upon the days of the week.
#### However, the polynomial model performs noticeably better than the single dimensional model when trained with data from the weekend days. For this reason we will use the polynomial model for the days of the weekend based model.
## Cross validation of each model will now be performed.

## Evaluation with cross-validation - single dimensional based model.

In [ ]:
# sorted(metrics.SCORERS.keys())

In [ ]:
one_d_scores = -cross_val_score(LinearRegression(), X, y, scoring='neg_mean_absolute_error', cv=5)
one_d_scores

In [ ]:
one_d_metrics = ['neg_mean_absolute_error', 'neg_mean_squared_error', 'r2']
one_d_scores = cross_validate(LinearRegression(), X, y, scoring=one_d_metrics, cv=5)
print("Metrics on cross validated one dimensional based model:")
one_d_scores

## Evaluation with cross-validation - polynomial based model.

In [ ]:
polynomial_scores = -cross_val_score(LinearRegression(), X_poly, y, scoring='neg_mean_absolute_error', cv=5)
polynomial_scores

In [ ]:
polynomial_metrics = ['neg_mean_absolute_error', 'neg_mean_squared_error', 'r2']
polynomial_scores = cross_validate(LinearRegression(), X_poly, y, scoring=polynomial_metrics, cv=5)
print("Metrics on cross validated polynomial based model:")
polynomial_scores

In [ ]:
sorted(scores.keys())